In [4]:
import pandas as pd

In [9]:
df = pd.read_csv('/content/input.csv')

In [10]:
df.columns

Index(['Employee Code', 'Manager Employee Code', 'Date of Joining',
       'Date of Exit', 'Compensation', 'Compensation 1', 'Compensation 1 date',
       'Compensation 2', 'Compensation 2 date', 'Review 1', 'Review 1 date',
       'Review 2', 'Review 2 date', 'Engagement 1', 'Engagement 1 date',
       'Engagement 2', 'Engagement 2 date'],
      dtype='object')

In [11]:
df.head(5)

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01


In [14]:
import pandas as pd
from datetime import datetime, timedelta

def parse_date(date_str):
    if pd.notna(date_str):
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    return None

def process_employee_data(input_file, output_file):
    df = pd.read_csv(input_file)

    employee_records = {}
    for _, row in df.iterrows():
        employee_code = row['Employee Code']
        if employee_code not in employee_records:
            employee_records[employee_code] = []

        # Process compensation data
        compensation_dates = [parse_date(row['Compensation 1 date']), parse_date(row['Compensation 2 date'])]
        compensations = [row['Compensation'], row['Compensation 1'], row['Compensation 2']]
        for compensation, date in zip(compensations, compensation_dates):
            if pd.notna(compensation):
                employee_records[employee_code].append({'Effective Date': date, 'Compensation': compensation})

        # Process review data
        review_dates = [parse_date(row['Review 1 date']), parse_date(row['Review 2 date'])]
        reviews = [row['Review 1'], row['Review 2']]
        for review, date in zip(reviews, review_dates):
            if pd.notna(review):
                employee_records[employee_code].append({'Effective Date': date, 'Review': review})

        # Process engagement data
        engagement_dates = [parse_date(row['Engagement 1 date']), parse_date(row['Engagement 2 date'])]
        engagements = [row['Engagement 1'], row['Engagement 2']]
        for engagement, date in zip(engagements, engagement_dates):
            if pd.notna(engagement):
                employee_records[employee_code].append({'Effective Date': date, 'Engagement': engagement})

    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['Employee Code', 'Manager Employee Code', 'Last Compensation', 'Compensation', 'Last Pay Raise Date',
                      'Variable Pay', 'Tenure in Org', 'Performance Rating', 'Engagement Score', 'Effective Date', 'End Date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Write historical records to CSV
        for employee_code, records in employee_records.items():
            records.sort(key=lambda x: x['Effective Date'])

            for i, record in enumerate(records):
                effective_date = record['Effective Date']
                if i < len(records) - 1:
                    end_date = records[i + 1]['Effective Date'] - timedelta(days=1)
                else:
                    end_date = datetime(2100, 1, 1).date()

                writer.writerow({'Employee Code': employee_code,
                                 'Manager Employee Code': row['Manager Employee Code'],
                                 'Last Compensation': '',  # Placeholder for now
                                 'Compensation': record.get('Compensation', ''),
                                 'Last Pay Raise Date': '',  # Placeholder for now
                                 'Variable Pay': '',  # Placeholder for now
                                 'Tenure in Org': '',  # Placeholder for now
                                 'Performance Rating': record.get('Review', ''),
                                 'Engagement Score': record.get('Engagement', ''),
                                 'Effective Date': effective_date,
                                 'End Date': end_date})

if __name__ == "__main__":
    input_file = "/content/input.csv"
    output_file = "output.csv"
    process_employee_data(input_file, output_file)


In [15]:
pd.read_csv('/content/output.csv')

,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,1,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,2100-01-01
1,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2021-03-01,2021-05-31
2,2,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,2021-06-01,2021-12-31
3,2,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,2022-01-01,2022-02-28
4,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2022-03-01,2022-05-31
5,2,1.0,NaN,NaN,NaN,NaN,NaN,9.5,NaN,2022-06-01,2022-12-31
6,2,1.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,2023-01-01,2100-01-01
7,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2021-03-01,2021-05-31
8,3,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,2021-06-01,2021-12-31
9,3,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,2022-01-01,2022-02-28


In [6]:
pd.read_csv('/content/output.csv')

,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date


In [7]:
import pandas as pd
from datetime import datetime, timedelta
import csv

def parse_date(date_str):
    if pd.notna(date_str):
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    return None

def process_employee_data(input_file, output_file):
    df = pd.read_csv(input_file)

    employee_records = {}
    for _, row in df.iterrows():
        employee_code = row['Employee Code']
        if employee_code not in employee_records:
            employee_records[employee_code] = []

        # Process compensation data
        compensation_dates = [parse_date(row['Compensation 1 date']), parse_date(row['Compensation 2 date'])]
        compensations = [row['Compensation'], row['Compensation 1'], row['Compensation 2']]
        last_compensation = None
        for compensation, date in zip(compensations, compensation_dates):
            if pd.notna(compensation):
                last_compensation = compensation
            employee_records[employee_code].append({'Effective Date': date, 'Compensation': last_compensation})

        # Process review data
        review_dates = [parse_date(row['Review 1 date']), parse_date(row['Review 2 date'])]
        reviews = [row['Review 1'], row['Review 2']]
        last_review = None
        for review, date in zip(reviews, review_dates):
            if pd.notna(review):
                last_review = review
            employee_records[employee_code].append({'Effective Date': date, 'Review': last_review})

        # Process engagement data
        engagement_dates = [parse_date(row['Engagement 1 date']), parse_date(row['Engagement 2 date'])]
        engagements = [row['Engagement 1'], row['Engagement 2']]
        last_engagement = None
        for engagement, date in zip(engagements, engagement_dates):
            if pd.notna(engagement):
                last_engagement = engagement
            employee_records[employee_code].append({'Effective Date': date, 'Engagement': last_engagement})

    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['Employee Code', 'Manager Employee Code', 'Last Compensation', 'Compensation', 'Last Pay Raise Date',
                      'Variable Pay', 'Tenure in Org', 'Performance Rating', 'Engagement Score', 'Effective Date', 'End Date']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # Write historical records to CSV
        for employee_code, records in employee_records.items():
            records.sort(key=lambda x: x['Effective Date'])

            last_review = None
            last_engagement = None
            for i, record in enumerate(records):
                effective_date = record['Effective Date']
                if i < len(records) - 1:
                    end_date = records[i + 1]['Effective Date'] - timedelta(days=1)
                else:
                    end_date = datetime(2100, 1, 1).date()

                compensation = record.get('Compensation', '')
                review = record.get('Review', last_review)
                engagement = record.get('Engagement', last_engagement)

                writer.writerow({'Employee Code': employee_code,
                                 'Manager Employee Code': row['Manager Employee Code'],
                                 'Last Compensation': '',  # Placeholder for now
                                 'Compensation': compensation,
                                 'Last Pay Raise Date': '',  # Placeholder for now
                                 'Variable Pay': '',  # Placeholder for now
                                 'Tenure in Org': '',  # Placeholder for now
                                 'Performance Rating': review,
                                 'Engagement Score': engagement,
                                 'Effective Date': effective_date,
                                 'End Date': end_date})

                if compensation:
                    last_compensation = compensation
                if review:
                    last_review = review
                if engagement:
                    last_engagement = engagement

if __name__ == "__main__":
    input_file = "/content/input.csv"
    output_file = "output.csv"process_employee_data(input_file, output_fi
    le)


TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [8]:
pd.read_csv('/content/input.csv')

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01


In [9]:
import pandas as pd
from datetime import datetime, timedelta
import csv

def parse_date(date_str):
    if pd.notna(date_str):
        return datetime.strptime(date_str, "%Y-%m-%d").date()
    return None

# Read input CSV file
input_file = "input.csv"
df = pd.read_csv(input_file)

# Initialize a dictionary to store employee records
employee_records = {}

# Process each row of the DataFrame
for _, row in df.iterrows():
    employee_code = row['Employee Code']
    if employee_code not in employee_records:
        employee_records[employee_code] = []

    # Process compensation data
    compensation_dates = [parse_date(row['Compensation 1 date']), parse_date(row['Compensation 2 date'])]
    compensations = [row['Compensation'], row['Compensation 1'], row['Compensation 2']]
    for compensation, date in zip(compensations, compensation_dates):
        if pd.notna(compensation):
            employee_records[employee_code].append({'Effective Date': date, 'Compensation': compensation})

    # Process review data
    review_dates = [parse_date(row['Review 1 date']), parse_date(row['Review 2 date'])]
    reviews = [row['Review 1'], row['Review 2']]
    for review, date in zip(reviews, review_dates):
        if pd.notna(review):
            employee_records[employee_code].append({'Effective Date': date, 'Review': review})

    # Process engagement data
    engagement_dates = [parse_date(row['Engagement 1 date']), parse_date(row['Engagement 2 date'])]
    engagements = [row['Engagement 1'], row['Engagement 2']]
    for engagement, date in zip(engagements, engagement_dates):
        if pd.notna(engagement):
            employee_records[employee_code].append({'Effective Date': date, 'Engagement': engagement})

# Write transformed data to output CSV file
output_file = "output.csv"
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['Employee Code', 'Manager Employee Code', 'Last Compensation', 'Compensation', 'Last Pay Raise Date',
                  'Variable Pay', 'Tenure in Org', 'Performance Rating', 'Engagement Score', 'Effective Date', 'End Date']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Write historical records to CSV
    for employee_code, records in employee_records.items():
        records.sort(key=lambda x: x['Effective Date'])

        for i, record in enumerate(records):
            effective_date = record['Effective Date']
            if i < len(records) - 1:
                end_date = records[i + 1]['Effective Date'] - timedelta(days=1)
            else:
                end_date = datetime(2100, 1, 1).date()

            writer.writerow({'Employee Code': employee_code,
                             'Manager Employee Code': row['Manager Employee Code'],
                             'Last Compensation': '',  # Placeholder for now
                             'Compensation': record.get('Compensation', ''),
                             'Last Pay Raise Date': '',  # Placeholder for now
                             'Variable Pay': '',  # Placeholder for now
                             'Tenure in Org': '',  # Placeholder for now
                             'Performance Rating': record.get('Review', ''),
                             'Engagement Score': record.get('Engagement', ''),
                             'Effective Date': effective_date,
                             'End Date': end_date})

print("Output file has been created:", output_file)


Output file has been created: output.csv


In [10]:
pd.read_csv('/content/output.csv')

,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,1,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,2100-01-01
1,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2021-03-01,2021-05-31
2,2,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,2021-06-01,2021-12-31
3,2,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,2022-01-01,2022-02-28
4,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2022-03-01,2022-05-31
5,2,1.0,NaN,NaN,NaN,NaN,NaN,9.5,NaN,2022-06-01,2022-12-31
6,2,1.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,2023-01-01,2100-01-01
7,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2021-03-01,2021-05-31
8,3,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,2021-06-01,2021-12-31
9,3,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,2022-01-01,2022-02-28


In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/input.csv')

In [9]:
df

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation 1,Compensation 1 date,Compensation 2,Compensation 2 date,Review 1,Review 1 date,Review 2,Review 2 date,Engagement 1,Engagement 1 date,Engagement 2,Engagement 2 date
0,1,NaN,2021-01-01,NaN,20000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1.0,2021-01-01,NaN,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,20000,10000.0,2022-01-01,20000.0,2023-01-01,9.0,2021-06-01,9.5,2022-06-01,4.0,2021-03-01,5.0,2022-03-01


In [3]:
df2 = pd.read_csv('/content/output.csv')

In [4]:
df2

,Employee Code,Manager Employee Code,Last Compensation,Compensation,Last Pay Raise Date,Variable Pay,Tenure in Org,Performance Rating,Engagement Score,Effective Date,End Date
0,1,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,NaN,2100-01-01
1,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2021-03-01,2021-05-31
2,2,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,2021-06-01,2021-12-31
3,2,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,2022-01-01,2022-02-28
4,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2022-03-01,2022-05-31
5,2,1.0,NaN,NaN,NaN,NaN,NaN,9.5,NaN,2022-06-01,2022-12-31
6,2,1.0,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,2023-01-01,2100-01-01
7,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,2021-03-01,2021-05-31
8,3,1.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,2021-06-01,2021-12-31
9,3,1.0,NaN,20000.0,NaN,NaN,NaN,NaN,NaN,2022-01-01,2022-02-28


In [11]:

# Filter out employee data
employee_data = df[df['Employee Code'].isin([2, 3])]

# Initialize empty DataFrame for transformed data
transformed_data = pd.DataFrame()

# Iterate over each row in employee data
for index, row in employee_data.iterrows():
    employee_code = row['Employee Code']
    manager_code = row['Manager Employee Code']
    joining_date = row['Date of Joining']
    exit_date = row['Date of Exit'] if not pd.isna(row['Date of Exit']) else '2100-01-01'

    for i in range(1, 3):
        compensation = row[f'Compensation {i}']
        compensation_date = row[f'Compensation {i} date']
        review_score = row[f'Review {i}']
        review_date = row[f'Review {i} date']
        engagement_score = row[f'Engagement {i}']
        engagement_date = row[f'Engagement {i} date']

        # Append transformed data to DataFrame
        transformed_data = transformed_data.append({
            'Employee Code': employee_code,
            'Manager Employee Code': manager_code,
            'Date of Joining': joining_date,
            'Date of Exit': exit_date,
            'Compensation': compensation,
            'Compensation Date': compensation_date,
            'Review Score': review_score,
            'Review Date': review_date,
            'Engagement Score': engagement_score,
            'Engagement Date': engagement_date
        }, ignore_index=True)

# Write the transformed data to a new CSV file
output_file = 'transformed_data.csv'
transformed_data.to_csv(output_file, index=False)

print("Transformation complete. Output saved to", output_file)


Transformation complete. Output saved to transformed_data.csv


<ipython-input-11-cb650342e0d5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append({
<ipython-input-11-cb650342e0d5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append({
<ipython-input-11-cb650342e0d5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append({
<ipython-input-11-cb650342e0d5>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_data = transformed_data.append({


In [12]:
pd.read_csv('/content/transformed_data.csv')

,Employee Code,Manager Employee Code,Date of Joining,Date of Exit,Compensation,Compensation Date,Review Score,Review Date,Engagement Score,Engagement Date
0,2,1.0,2021-01-01,2100-01-01,10000.0,2022-01-01,9.0,2021-06-01,4.0,2021-03-01
1,2,1.0,2021-01-01,2100-01-01,20000.0,2023-01-01,9.5,2022-06-01,5.0,2022-03-01
2,3,1.0,2021-01-01,2023-12-31,10000.0,2022-01-01,9.0,2021-06-01,4.0,2021-03-01
3,3,1.0,2021-01-01,2023-12-31,20000.0,2023-01-01,9.5,2022-06-01,5.0,2022-03-01
